Распаковка выборки

In [ ]:
!unzip limb.zip
!unzip perlimb.zip

Archive:  limb.zip
replace limb/convert.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

### InceptionResNetV2 perelimb начальное обучение

In [25]:
import tensorflow as tf
from keras.applications import InceptionResNetV2
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

# Задайте размер изображений

train_path = "perelimb/train"  # путь для обучающей выборки
test_path = "perelimb/test"
# Создайте генераторы данных для обучения и проверки
train_datagen = ImageDataGenerator(
rescale=1.0/255,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

img_width, img_height = 499, 499
batch_size = 16
train_generator = train_datagen.flow_from_directory(
train_path,
target_size=(img_width, img_height),
batch_size=batch_size,
class_mode="binary"
)

test_generator = test_datagen.flow_from_directory(
test_path,
target_size=(img_width, img_height),
batch_size=batch_size,
class_mode="binary"
)

# Загрузите предобученную модель InceptionResNetV2
InceptionResNetV2 = InceptionResNetV2(weights="imagenet", include_top=False, input_shape=(img_width, img_height, 3))
callback = ModelCheckpoint(filepath="InceptionResNetV2_model.h5", monitor="val_accuracy", save_best_only=True)
# Добавьте свои слои поверх InceptionResNetV2
x = InceptionResNetV2.output
x = GlobalAveragePooling2D()(x)
x = Dense(516, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)  # добавим dropout для предотвращения переобучения
#x = Dense(256, activation="relu")(x)
#x = tf.keras.layers.Dropout(0.5)(x)  # добавим dropout для предотвращения переобучения
output = Dense(1, activation="sigmoid")(x)

# Создайте модель на базе InceptionResNetV2
model = Model(inputs=InceptionResNetV2.input, outputs=output)

# Заморозьте веса предобученных слоев
for layer in InceptionResNetV2.layers:
 layer.trainable = False

# Скомпилируйте и обучите модель
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

model.fit(train_generator, epochs=15, steps_per_epoch=len(train_generator), validation_data=test_generator, validation_steps=len(test_generator),callbacks=[callback])

accuracy = model.evaluate(test_generator)

print("Результат = ", accuracy[1])



Found 190 images belonging to 2 classes.
Found 69 images belonging to 2 classes.
Epoch 1/15
12/12 [==============================] - ETA: 0s - loss: 0.8365 - accuracy: 0.5105

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


12/12 [==============================] - 35s 2s/step - loss: 0.8365 - accuracy: 0.5105 - val_loss: 0.6306 - val_accuracy: 0.6377
Epoch 2/15
12/12 [==============================] - 16s 1s/step - loss: 0.6531 - accuracy: 0.5737 - val_loss: 0.6431 - val_accuracy: 0.6377
Epoch 3/15
12/12 [==============================] - 23s 2s/step - loss: 0.6200 - accuracy: 0.6211 - val_loss: 0.5460 - val_accuracy: 0.7246
Epoch 4/15
12/12 [==============================] - 22s 2s/step - loss: 0.5823 - accuracy: 0.6684 - val_loss: 0.5785 - val_accuracy: 0.6377
Epoch 5/15
12/12 [==============================] - 15s 1s/step - loss: 0.5865 - accuracy: 0.6947 - val_loss: 0.5232 - val_accuracy: 0.7246
Epoch 6/15
12/12 [==============================] - 15s 1s/step - loss: 0.5984 - accuracy: 0.6316 - val_loss: 0.5160 - val_accuracy: 0.7246
Epoch 7/15
12/12 [==============================] - 15s 1s/step - loss: 0.5454 - accuracy: 0.7000 - val_loss: 0.5160 - val_accuracy: 0.7246
Epoch 8/15
12/12 [=============

### InceptionResNetV2 perelimb fine tuning




In [27]:
import tensorflow as tf
from keras.applications import InceptionResNetV2
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

# Задайте размер изображений

train_path = "perelimb/train"  # путь для обучающей выборки
test_path = "perelimb/test"
# Создайте генераторы данных для обучения и проверки
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

img_width, img_height = 499, 499
batch_size = 8
train_generator = train_datagen.flow_from_directory(
train_path,
target_size=(img_width, img_height),
batch_size=batch_size,
class_mode="binary"
)

test_generator = test_datagen.flow_from_directory(
test_path,
target_size=(img_width, img_height),
batch_size=batch_size,
class_mode="binary"
)

# Загрузите предобученную модель InceptionResNetV2
InceptionResNetV2 = InceptionResNetV2(weights=None, include_top=False, input_shape=(img_width, img_height, 3))
callback = ModelCheckpoint(filepath="InceptionResNetV2_perelimb_model3.h5", monitor="val_accuracy", save_best_only=True)
# Добавьте свои слои поверх InceptionResNetV2
x = InceptionResNetV2.output
x = GlobalAveragePooling2D()(x)
x = Dense(516, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)  # добавим dropout для предотвращения переобучения
#x = Dense(256, activation="relu")(x)
#x = tf.keras.layers.Dropout(0.5)(x)  # добавим dropout для предотвращения переобучения
output = Dense(1, activation="sigmoid")(x)


# Создайте модель на базе InceptionResNetV2
model = Model(inputs=InceptionResNetV2.input, outputs=output)
model.load_weights("InceptionResNetV2_model.h5")

model.summary()
model.compile(optimizer=Adam(1e-4), loss="binary_crossentropy", metrics=["accuracy"])

model.fit(train_generator, epochs=15, steps_per_epoch=len(train_generator), validation_data=test_generator, validation_steps=len(test_generator),callbacks=[callback])

accuracy = model.evaluate(test_generator)

print("Результат fine tune'd = ", accuracy[1])
model.save("InceptionResNetV2_perelimb.keras")

Found 190 images belonging to 2 classes.
Found 69 images belonging to 2 classes.
Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_15 (InputLayer)       [(None, 499, 499, 3)]        0         []                            
                                                                                                  
 conv2d_649 (Conv2D)         (None, 249, 249, 32)         864       ['input_15[0][0]']            
                                                                                                  
 batch_normalization_649 (B  (None, 249, 249, 32)         96        ['conv2d_649[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activatio

### InceptionResNetV2 perelimb финальная прогонка

In [28]:
from os import listdir

import matplotlib.pyplot as plt
import keras.utils as image
from keras.models import load_model


def result(
    dir, true, pred1, pred2, model
):  # функция, выводящая результат распознавания каждого изображения в каждой папке контрольной выборки
    count_bolen = 0
    count_zdorov = 0
    img_dir = listdir(dir)
    for file in img_dir:  # перебор файлов в папке
        img = image.load_img(
            dir + "/" + file, target_size=(499, 499)
        )  # загрузка изображения
        x = image.img_to_array(img)
        x = x.reshape(1, 499, 499, 3)
        x /= 255
        prediction = model.predict(x)  # обработка нейросетью

        if prediction[0][0] < (
            0.5
        ):  # вывод результата в зависимости от успешности распознавания
            if true == "zdorov":
                count_zdorov += 1
                # счётчик ошибок "здоровых" (якобы больных)
                show(pred1, prediction[0][0], img)  # болен
        else:
            if true == "bolen":
                count_bolen += 1  # счётчик ошибок "больных" (якобы здоровых)
                show(pred2, prediction[0][0], img)  # здоров
            show(pred2, prediction[0][0], img)

    print("Якобы здоровых: ", count_bolen)
    print("Якобы больных: ", count_zdorov)


def show(title, pred, img):  # функция показа результата распознавания
    plt.imshow(img.convert("RGBA"))
    plt.title(title + str(pred))
    plt.show()


model = load_model("InceptionResNetV2_perelimb.keras")

result("perelimb/test/bolen", "bolen", "Патология -> ", "Якобы условная норма -> ", model)
result("perelimb/test/zdorov", "zdorov", "Якобы патология -> ", "Условная норма -> ", model)


Output hidden; open in https://colab.research.google.com to view.

### InceptionResNetV2 limb начальное обучение




In [29]:
import tensorflow as tf
from keras.applications import InceptionResNetV2
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

# Задайте размер изображений

train_path = "limb/train"  # путь для обучающей выборки
test_path = "limb/test"
# Создайте генераторы данных для обучения и проверки
train_datagen = ImageDataGenerator(
rescale=1.0/255,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

img_width, img_height = 499, 499
batch_size = 16
train_generator = train_datagen.flow_from_directory(
train_path,
target_size=(img_width, img_height),
batch_size=batch_size,
class_mode="binary"
)

test_generator = test_datagen.flow_from_directory(
test_path,
target_size=(img_width, img_height),
batch_size=batch_size,
class_mode="binary"
)

# Загрузите предобученную модель InceptionResNetV2
InceptionResNetV2 = InceptionResNetV2(weights="imagenet", include_top=False, input_shape=(img_width, img_height, 3))
callback = ModelCheckpoint(filepath="InceptionResNetV2_limb_model.h5", monitor="val_accuracy", save_best_only=True)
# Добавьте свои слои поверх InceptionResNetV2
x = InceptionResNetV2.output
x = GlobalAveragePooling2D()(x)
x = Dense(516, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)  # добавим dropout для предотвращения переобучения
x = Dense(256, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)  # добавим dropout для предотвращения переобучения
output = Dense(1, activation="sigmoid")(x)

# Создайте модель на базе InceptionResNetV2
model = Model(inputs=InceptionResNetV2.input, outputs=output)

# Заморозьте веса предобученных слоев
for layer in InceptionResNetV2.layers:
 layer.trainable = False

# Скомпилируйте и обучите модель
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

model.fit(train_generator, epochs=15, steps_per_epoch=len(train_generator), validation_data=test_generator, validation_steps=len(test_generator),callbacks=[callback])

accuracy = model.evaluate(test_generator)

print("Результат = ", accuracy[1])



Found 339 images belonging to 2 classes.
Found 120 images belonging to 2 classes.
Epoch 1/15
22/22 [==============================] - 63s 2s/step - loss: 0.7662 - accuracy: 0.5811 - val_loss: 0.5540 - val_accuracy: 0.7250
Epoch 2/15
22/22 [==============================] - 28s 1s/step - loss: 0.7063 - accuracy: 0.6283 - val_loss: 0.5334 - val_accuracy: 0.7250
Epoch 3/15
22/22 [==============================] - 27s 1s/step - loss: 0.6924 - accuracy: 0.5929 - val_loss: 0.5245 - val_accuracy: 0.7250
Epoch 4/15
22/22 [==============================] - 35s 2s/step - loss: 0.6998 - accuracy: 0.6195 - val_loss: 0.5176 - val_accuracy: 0.7417
Epoch 5/15
22/22 [==============================] - 27s 1s/step - loss: 0.6419 - accuracy: 0.6342 - val_loss: 0.5170 - val_accuracy: 0.7250
Epoch 6/15
22/22 [==============================] - 36s 2s/step - loss: 0.6062 - accuracy: 0.6637 - val_loss: 0.5200 - val_accuracy: 0.7667
Epoch 7/15
22/22 [==============================] - 30s 1s/step - loss: 0.6060

### InceptionResNetV2 limb fine tuning




In [30]:
import tensorflow as tf
from keras.applications import InceptionResNetV2
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

# Задайте размер изображений

train_path = "limb/train"  # путь для обучающей выборки
test_path = "limb/test"
# Создайте генераторы данных для обучения и проверки
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

img_width, img_height = 499, 499
batch_size = 8
train_generator = train_datagen.flow_from_directory(
train_path,
target_size=(img_width, img_height),
batch_size=batch_size,
class_mode="binary"
)

test_generator = test_datagen.flow_from_directory(
test_path,
target_size=(img_width, img_height),
batch_size=batch_size,
class_mode="binary"
)

# Загрузите предобученную модель InceptionResNetV2
InceptionResNetV2 = InceptionResNetV2(weights=None, include_top=False, input_shape=(img_width, img_height, 3))
callback = ModelCheckpoint(filepath="InceptionResNetV2_limb_model_tuned.h5", monitor="val_accuracy", save_best_only=True)
# Добавьте свои слои поверх InceptionResNetV2
x = InceptionResNetV2.output
x = GlobalAveragePooling2D()(x)
x = Dense(516, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)  # добавим dropout для предотвращения переобучения
x = Dense(256, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)  # добавим dropout для предотвращения переобучения
output = Dense(1, activation="sigmoid")(x)


# Создайте модель на базе InceptionResNetV2
model = Model(inputs=InceptionResNetV2.input, outputs=output)
model.load_weights("InceptionResNetV2_limb_model.h5")

model.summary()
model.compile(optimizer=Adam(1e-4), loss="binary_crossentropy", metrics=["accuracy"])

model.fit(train_generator, epochs=15, steps_per_epoch=len(train_generator), validation_data=test_generator, validation_steps=len(test_generator),callbacks=[callback])

accuracy = model.evaluate(test_generator)

print("Результат fine tune'd = ", accuracy[1])
model.save("InceptionResNetV2_limb.keras")

Found 339 images belonging to 2 classes.
Found 120 images belonging to 2 classes.
Model: "model_16"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_17 (InputLayer)       [(None, 499, 499, 3)]        0         []                            
                                                                                                  
 conv2d_1055 (Conv2D)        (None, 249, 249, 32)         864       ['input_17[0][0]']            
                                                                                                  
 batch_normalization_1055 (  (None, 249, 249, 32)         96        ['conv2d_1055[0][0]']         
 BatchNormalization)                                                                              
                                                                                                  
 activati

### InceptionResNetV2 limb финальная прогонка




In [31]:
from os import listdir

import matplotlib.pyplot as plt
import keras.utils as image
from keras.models import load_model


def result(
    dir, true, pred1, pred2, model
):  # функция, выводящая результат распознавания каждого изображения в каждой папке контрольной выборки
    count_bolen = 0
    count_zdorov = 0
    img_dir = listdir(dir)
    for file in img_dir:  # перебор файлов в папке
        img = image.load_img(
            dir + "/" + file, target_size=(499, 499)
        )  # загрузка изображения
        x = image.img_to_array(img)
        x = x.reshape(1, 499, 499, 3)
        x /= 255
        prediction = model.predict(x)  # обработка нейросетью

        if prediction[0][0] < (
            0.5
        ):  # вывод результата в зависимости от успешности распознавания
            if true == "zdorov":
                count_zdorov += 1
                # счётчик ошибок "здоровых" (якобы больных)
                show(pred1, prediction[0][0], img)  # болен
        else:
            if true == "bolen":
                count_bolen += 1  # счётчик ошибок "больных" (якобы здоровых)
                show(pred2, prediction[0][0], img)  # здоров
            show(pred2, prediction[0][0], img)

    print("Якобы здоровых: ", count_bolen)
    print("Якобы больных: ", count_zdorov)


def show(title, pred, img):  # функция показа результата распознавания
    plt.imshow(img.convert("RGBA"))
    plt.title(title + str(pred))
    plt.show()


model = load_model("InceptionResNetV2_limb.keras")

result("limb/test/bolen", "bolen", "Патология -> ", "Якобы условная норма -> ", model)
result("limb/test/zdorov", "zdorov", "Якобы патология -> ", "Условная норма -> ", model)


Output hidden; open in https://colab.research.google.com to view.